## 无降维
### 没有使用sklearn库，自己手写k-means算法

In [3]:
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


def get_distances(X, k, centroids):
    n_samples, n_features = X.shape
    
    distances = np.zeros([X.shape[0], centroids.shape[0]])
    for i, x in enumerate(X):
        for j, centroid in enumerate(centroids):
            b = sum(np.sqrt((x - centroid)**2))
            distances[i][j] = b
    
    return distances

# 定义k-means聚类函数
def k_means(X, k, max_iters, callback=None):
    n_samples, n_features = X.shape
    
    # 随机初始化聚类中心, centroids是聚类中心
    centroids = X[np.random.choice(n_samples, k, replace=False), :]
    
    labels = []

    # 迭代更新聚类中心
    for i in range(max_iters):
        # 计算每个样本到聚类中心的距离
        distances = get_distances(X, k, centroids)

        # 分配每个样本到最近的聚类中心
        # labels存储的是每个样本离的最近的聚类中心的索引
        # labels的索引相当于每个样本的索引
        labels = []
        loss = 0 # 误差
        for i, distance in enumerate(distances):
            d = 0x3f3f3f3f
            index = 0
            for j, dist in enumerate(distance):
                # print("dist ", dist)
                if dist < d:
                    index = j
                    d = dist
            
            # print(d)
            loss += d
            labels.append(index)

        if callback is not None:
            # 通过这个回调函数求每次迭代的时候的损失
            # 直接加上每个样本到他聚类中心的距离
            callback(loss)

        # 更新聚类中心
        centroids = updataCenterIds(k, labels, centroids)

    return labels, centroids

def updataCenterIds(k, labels, centroids):

    # 这个版本时间复杂度为 O(k*X.shape[0])
    for i in range(k):
        # 存储到这个聚类中心最近的样本
        min_pointer = []
        for j, label in enumerate(labels):
            if label == i:
                min_pointer.append(X[j])
        
        centroids[i] = np.sum(min_pointer, axis=0) / len(min_pointer)

    return centroids



In [ ]:
# 读取训练集数据
with open('./MNIST_data/train-images.idx3-ubyte', 'rb') as f:
    train_X = np.frombuffer(f.read(), dtype=np.uint8, offset=16).reshape(-1, 28*28)

max_iters = 100

k = 10

X = train_X[:20000]

labels, centroids = k_means(X, k, max_iters)
print("labels", len(labels))
print("centroids", centroids)

### sklearn库的k-means算法

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=42, max_iter= 50)
kmeans.fit(X)

## 降维之后
### 先计算降到多少维可以保持80%的方差贡献率

#### 数据预处理

In [ ]:
#求784维的每一维的均值
xi = train_X.shape[1]
Xmean = np.zeros(xi,dtype="f")#.reshape(xi)
Xstd = np.zeros(xi,dtype="f")
#规范化的样本数据
Strain_X = np.zeros(len(train_X) * xi, dtype="f").reshape(xi, len(train_X))#784*55000

print(Strain_X.shape)

for i in range(xi):#784
    Xmean[i] = np.mean(train_X[:,i]) #求第i维向量的所有数据点的均值
    Xstd[i] = np.std(train_X[:,i], ddof=1) #求第i维向量的所有数据点的标准差

print(len(train_X))

# #规范化所有样本数据
for i in range(len(train_X)):
    for j in range(xi):
        # print(train_X[i][j].T-Xmean[j])
        Strain_X[j][i] = (train_X[i][j].T-Xmean[j])#不归一化
Strain_X

##### 样本相关矩阵(均值为零，方差不为1) 784*784(协方差矩阵)

In [ ]:
R = (1 / len(train_X) - 1) * (np.dot(Strain_X, Strain_X.T))
R

##### 奇异值分解求特征值

In [ ]:
import math
# 奇异值分解
U,S,VT =np.linalg.svd(R) #VT 的 i 行对应 i 个特征值的特征向量

# 奇异值开根求特征值
S1=np.zeros(len(S)).reshape(len(S),1)
for i in range(len(S)):
    S1[i] = math.sqrt(S[i])
sorted(S1, reverse=True)
S1

##### 求累计方差贡献率

In [ ]:
miuk = 0.8
addS = 0
#记录累加方程贡献率在大于miuk时，有多少个主成分
cnt = 0

for i in range(len(S)):
    addS += S1[i]
    cnt += 1
    if((addS / sum(S1)) >= miuk):
        cnt = i
        break
print(cnt)

# 进行降维
pca =PCA(n_components =cnt + 1)
pca.fit(train_X)
X=pca.transform(train_X)# 降维后的结果

### 通过自己手写的k-means算法进行聚类，并且与没有降维之前作比较

In [ ]:
# 原本的60000个样本点太多了，跑不出来，少选几个
X = X[:20000]

labels, centroids = k_means(X, k, max_iters)
print("labels", len(labels))
print("centroids", centroids)

### 使用k-means对降维之后的数据进行聚类

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=42, max_iter= 50)
kmeans.fit(X)